In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
!pip install pgmpy


In [8]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import textwrap

class MysterySolver:
    def __init__(self):
        # Define the Bayesian Network structure
        self.model = BayesianNetwork([
            ('Motive', 'DetectiveDanGuilty'),
            ('Motive', 'ProfessorPennyGuilty'),
            ('AlibiDan', 'DetectiveDanGuilty'),
            ('AlibiPenny', 'ProfessorPennyGuilty'),
            ('Evidence', 'DetectiveDanGuilty'),
            ('Evidence', 'ProfessorPennyGuilty')
        ])
        
        # Define Conditional Probability Distributions (CPDs)
        cpd_motive = TabularCPD(variable='Motive', variable_card=2, values=[[0.6], [0.4]])
        cpd_alibi_dan = TabularCPD(variable='AlibiDan', variable_card=2, values=[[0.9], [0.1]])
        cpd_alibi_penny = TabularCPD(variable='AlibiPenny', variable_card=2, values=[[0.8], [0.2]])
        cpd_evidence = TabularCPD(variable='Evidence', variable_card=2, values=[[0.5], [0.5]])

        # Suspect Dan's guilt based on motive, alibi, and evidence
        cpd_dan_guilty = TabularCPD(
            variable='DetectiveDanGuilty', variable_card=2,
            values=[
                [0.9, 0.6, 0.4, 0.2, 0.8, 0.5, 0.3, 0.1],  # Not guilty
                [0.1, 0.4, 0.6, 0.8, 0.2, 0.5, 0.7, 0.9]   # Guilty
            ],
            evidence=['Motive', 'AlibiDan', 'Evidence'],
            evidence_card=[2, 2, 2]
        )

        # Suspect Penny's guilt based on motive, alibi, and evidence
        cpd_penny_guilty = TabularCPD(
            variable='ProfessorPennyGuilty', variable_card=2,
            values=[
                [0.85, 0.5, 0.3, 0.1, 0.75, 0.4, 0.2, 0.05],  # Not guilty
                [0.15, 0.5, 0.7, 0.9, 0.25, 0.6, 0.8, 0.95]   # Guilty
            ],
            evidence=['Motive', 'AlibiPenny', 'Evidence'],
            evidence_card=[2, 2, 2]
        )

        # Add CPDs to the model
        self.model.add_cpds(cpd_motive, cpd_alibi_dan, cpd_alibi_penny, cpd_evidence, cpd_dan_guilty, cpd_penny_guilty)

        # Verify the model structure
        assert self.model.check_model()

        # Set up the inference engine
        self.inference = VariableElimination(self.model)

    def infer_suspects(self, motive, alibi_dan, alibi_penny, evidence):
        # Map inputs to Bayesian network values
        motive_value = 1 if motive else 0
        alibi_dan_value = 1 if alibi_dan else 0
        alibi_penny_value = 1 if alibi_penny else 0
        evidence_value = 1 if evidence else 0

        # Query the network for probabilities of each suspect being guilty
        result_dan = self.inference.query(variables=['DetectiveDanGuilty'], evidence={
            'Motive': motive_value, 'AlibiDan': alibi_dan_value, 'Evidence': evidence_value
        })
        result_penny = self.inference.query(variables=['ProfessorPennyGuilty'], evidence={
            'Motive': motive_value, 'AlibiPenny': alibi_penny_value, 'Evidence': evidence_value
        })

        # Display results
        print("\n### Mystery Results ###")
        print("Probability of Detective Dan being Guilty:")
        print(result_dan)
        print("\nProbability of Professor Penny being Guilty:")
        print(result_penny)


# Main application code
def main():
    # Story and Introduction
    story = textwrap.dedent("""
        Welcome to the Mystery Solver!
        
        A crime has been committed at the old library, and two primary suspects have emerged:
        - Detective Dan: Known for his sly nature and secretive background. He claims he was at the café.
        - Professor Penny: A distinguished scholar with a mysterious past. She says she was working late in her office.
        
        You have been given clues and must determine who might be guilty. Use your deductive skills!
    """)
    print(story)
    
    # Initialize the mystery solver
    solver = MysterySolver()
    
    # Collect evidence from the user
    motive = input("Do you believe there was a motive (yes/no)? ").strip().lower() == 'yes'
    alibi_dan = input("Detective Dan claims he was at the café. Does he have a solid alibi (yes/no)? ").strip().lower() == 'no'
    alibi_penny = input("Professor Penny says she was in her office. Is her alibi questionable (yes/no)? ").strip().lower() == 'yes'
    evidence = input("Was there physical evidence linking one of them to the crime scene (yes/no)? ").strip().lower() == 'yes'
    
    # Perform inference
    solver.infer_suspects(motive, alibi_dan, alibi_penny, evidence)

if __name__ == "__main__":
    main()



Welcome to the Mystery Solver!

A crime has been committed at the old library, and two primary suspects have emerged:
- Detective Dan: Known for his sly nature and secretive background. He claims he was at the café.
- Professor Penny: A distinguished scholar with a mysterious past. She says she was working late in her office.

You have been given clues and must determine who might be guilty. Use your deductive skills!



Do you believe there was a motive (yes/no)?  no
Detective Dan claims he was at the café. Does he have a solid alibi (yes/no)?  no
Professor Penny says she was in her office. Is her alibi questionable (yes/no)?  no
Was there physical evidence linking one of them to the crime scene (yes/no)?  yes



### Mystery Results ###
Probability of Detective Dan being Guilty:
+-----------------------+---------------------------+
| DetectiveDanGuilty    |   phi(DetectiveDanGuilty) |
+=======================+===========================+
| DetectiveDanGuilty(0) |                    0.2000 |
+-----------------------+---------------------------+
| DetectiveDanGuilty(1) |                    0.8000 |
+-----------------------+---------------------------+

Probability of Professor Penny being Guilty:
+-------------------------+-----------------------------+
| ProfessorPennyGuilty    |   phi(ProfessorPennyGuilty) |
+=========================+=============================+
| ProfessorPennyGuilty(0) |                      0.5000 |
+-------------------------+-----------------------------+
| ProfessorPennyGuilty(1) |                      0.5000 |
+-------------------------+-----------------------------+
